In [216]:
import pandas as pd
import numpy as np

# **1. Definición de variables necesarias para la extracción.**

In [217]:
# Defined variables

##Define league names and their IDs
dict_league_names = {'Premier-League': '9',
                     'Ligue-1': '13',
                     'Bundesliga': '20',
                     'Serie-A': '11',
                     'La-Liga': '12',
                     'Major-League-Soccer': '22',
                     'Big-5-European-Leagues': 'Big5'
                    }
## Define list of long names for 'Big 5' European Leagues and MLS
lst_league_names_long = ['Premier-League', 'Ligue-1', 'Bundesliga', 'Serie-A', 'La-Liga', 'Major-League-Soccer', 'Big-5-European-Leagues']

## Define seasons to scrape
lst_seasons = ['2017-2018', '2018-2019', '2019-2020', '2020-2021', '2021-2022']

# **2. Función para extracción de estádisticas de los jugadores**

In [218]:
# Define function for scraping a defined season and competition of FBref player data
def get_fbref_player_stats(lst_league_names, lst_seasons):
    
    ## Define list of league names
    league_names_long = lst_league_names
    
    ## Define seasons to scrape
    seasons = lst_seasons

    ## Scrape information for each player
    for season in seasons:

        ### Print message
        print(f'Scraping started for the {season} season...')

        ### Loop through leagues
        for league_name_long in league_names_long:
            league_name_short = [v for k,v in dict_league_names.items() if k == league_name_long][0]
    
            print(f'Scraping started for player stats data for {league_name_long} league for the {season} season...')
            
            print(f'Scraping Standard stats...')
            url_std_stats = f'https://widgets.sports-reference.com/wg.fcgi?css=1&site=fb&url=%2Fen%2Fcomps%2F{league_name_short}%2F{season}%2Fstats%2Fplayers%2F{season}-{league_name_long}&div=div_stats_standard'
            df_std_stats = pd.read_html(url_std_stats, header=1)[0]  
            
            ##### Shooting stats
            print(f'Scraping Shooting stats...')
            url_shooting = f'https://widgets.sports-reference.com/wg.fcgi?css=1&site=fb&url=%2Fen%2Fcomps%2F{league_name_short}%2F{season}%2Fshooting%2Fplayers%2F{season}-{league_name_long}&div=div_stats_shooting'
            df_shooting = pd.read_html(url_shooting, header=1)[0]

            ##### Passing stats
            print(f'Scraping Passing stats...')
            url_passing = f'https://widgets.sports-reference.com/wg.fcgi?css=1&site=fb&url=%2Fen%2Fcomps%2F{league_name_short}%2F{season}%2Fpassing%2Fplayers%2F{season}-{league_name_long}&div=div_stats_passing'
            df_passing = pd.read_html(url_passing, header=1)[0]

            ##### Pass Types stats
            print(f'Scraping Pass Types stats...')
            url_passing_types = f'https://widgets.sports-reference.com/wg.fcgi?css=1&site=fb&url=%2Fen%2Fcomps%2F{league_name_short}%2F{season}%2Fpassing_types%2Fplayers%2F{season}-{league_name_long}&div=div_stats_passing_types'
            df_passing_types = pd.read_html(url_passing_types, header=1)[0]

            ##### Goals and Shot Creation stats
            print(f'Scraping Goals and Shot Creation stats...')
            url_gca = f'https://widgets.sports-reference.com/wg.fcgi?css=1&site=fb&url=%2Fen%2Fcomps%2F{league_name_short}%2F{season}%2Fgca%2Fplayers%2F{season}-{league_name_long}&div=div_stats_gca'
            df_gca = pd.read_html(url_gca, header=1)[0]

            ##### Defensive Actions stats
            print(f'Scraping Defensive Actions stats...')
            url_defense = f'https://widgets.sports-reference.com/wg.fcgi?css=1&site=fb&url=%2Fen%2Fcomps%2F{league_name_short}%2F{season}%2Fdefense%2Fplayers%2F{season}-{league_name_long}&div=div_stats_defense'
            df_defense = pd.read_html(url_defense, header=1)[0]

            ##### Possession stats
            print(f'Scraping Possession stats...')
            url_possession = f'https://widgets.sports-reference.com/wg.fcgi?css=1&site=fb&url=%2Fen%2Fcomps%2F{league_name_short}%2F{season}%2Fpossession%2Fplayers%2F{season}-{league_name_long}&div=div_stats_possession'
            df_possession = pd.read_html(url_possession, header=1)[0]

            ##### Playing Time stats
            print(f'Scraping Playing Time stats...')
            url_playing_time = f'https://widgets.sports-reference.com/wg.fcgi?css=1&site=fb&url=%2Fen%2Fcomps%2F{league_name_short}%2F{season}%2Fplayingtime%2Fplayers%2F{season}-{league_name_long}&div=div_stats_playing_time'
            df_playing_time = pd.read_html(url_playing_time, header=1)[0]

            ##### Miscellaneous stats
            print(f'Scraping Miscellaneous stats...')
            url_misc = f'https://widgets.sports-reference.com/wg.fcgi?css=1&site=fb&url=%2Fen%2Fcomps%2F{league_name_short}%2F{season}%2Fmisc%2Fplayers%2F{season}-{league_name_long}&div=div_stats_misc'
            df_misc = pd.read_html(url_misc, header=1)[0]
      
            ##### Concatenate defined individual DataFrames
                
            ####### Define DataFrames to be concatenated side-by-side (not all of them)
            lst_dfs = [df_std_stats, df_shooting, df_passing, df_passing_types, df_gca, df_defense, df_possession]

            ###### Concatenate DataFrames side-by-side (indicated in list above)
            df_all = pd.concat(lst_dfs, axis=1)

            ###### Drop duplicate columns
            df_all = df_all.loc[:,~df_all.columns.duplicated()]

            ###### Drop duplicate rows
            df_all = df_all.drop_duplicates()
                
            ##### Left join defined individual DataFrames
                
            ####### Define join conditions
            conditions_join = ['Player', 'Nation', 'Pos', 'Squad', 'Comp']

            ###### Left join Playing Time data
            df_all = pd.merge(df_all, df_playing_time, left_on=conditions_join, right_on=conditions_join, how='left')

            ###### Remove duplicate columns after join (contain '_y') and remove '_x' suffix from kept columns
            df_all = df_all[df_all.columns.drop(list(df_all.filter(regex='_y')))]
            df_all.columns = df_all.columns.str.replace('_x','')
                
            ###### Drop duplicate rows
            df_all = df_all.drop_duplicates()

            ###### Left join Misc data
            df_all = pd.merge(df_all, df_misc, left_on=conditions_join, right_on=conditions_join, how='left')

            ###### Remove duplicate columns after join (contain '_y') and remove '_x' suffix from kept columns
            df_all = df_all[df_all.columns.drop(list(df_all.filter(regex='_y')))]
            df_all.columns = df_all.columns.str.replace('_x','')
                
            ###### Drop duplicate rows
            df_all = df_all.drop_duplicates()
                
                
            ##### FORMAT DataFrames
                
            ###### Take first two digits of age - fixes current season issue with extra values
            df_all['Age'] = df_all['Age'].astype(str).str[:2]
                
            ###### Create columns for league code and season
            df_all['League Name'] = league_name_long
            df_all['League ID'] = league_name_short
            df_all['Season'] = season          

            ###### Drop duplicates
            df_all = df_all.drop_duplicates()

    return df_all

# **3. Extracción de jugadores**
Se hace una extracción por temporada para evitar el time out del servidor.


In [219]:
# Leagues selected to scrape
lst_league_names = ['Big-5-European-Leagues']     #'Premier-League', 'Ligue-1', 'Bundesliga', 'Serie-A', 'La-Liga', 'Major-League-Soccer']

In [220]:
playerdata2022_2023 = get_fbref_player_stats(lst_league_names,['2022-2023'])

Scraping started for the 2022-2023 season...
Scraping started for player stats data for Big-5-European-Leagues league for the 2022-2023 season...
Scraping Standard stats...
Scraping Shooting stats...
Scraping Passing stats...
Scraping Pass Types stats...
Scraping Goals and Shot Creation stats...
Scraping Defensive Actions stats...
Scraping Possession stats...
Scraping Playing Time stats...
Scraping Miscellaneous stats...


# **4. Función para extracción de estádisticas de los porteros**

In [221]:
# Define function for scraping a defined season and competition of FBref player data
def get_fbref_goalkeeper_stats(lst_league_names, lst_seasons):
    
    ## Define list of league names
    league_names_long = lst_league_names
    
    ## Define seasons to scrape
    seasons = lst_seasons
    
    ## Scrape information for each player
    for season in seasons:

        ### Print message
        print(f'Scraping started for the {season} season...')

        ### Loop through leagues
        for league_name_long in league_names_long:
            league_name_short = [v for k,v in dict_league_names.items() if k == league_name_long][0]
        
            print(f'Scraping started for goalkeeper stats data for {league_name_long} league for the {season} season...')

            ##### Standard stats
            print(f'Scraping Standard stats...')
            url_std_stats = f'https://widgets.sports-reference.com/wg.fcgi?css=1&site=fb&url=%2Fen%2Fcomps%2F{league_name_short}%2F{season}%2Fstats%2Fplayers%2F{season}-{league_name_long}&div=div_stats_standard'
            df_std_stats = pd.read_html(url_std_stats, header=1)[0]

            ##### Goalkeeper stats
            print(f'Scraping Goalkeeper stats...')
            url_keepers = f'https://widgets.sports-reference.com/wg.fcgi?css=1&site=fb&url=%2Fen%2Fcomps%2F{league_name_short}%2F{season}%2Fkeepers%2Fplayers%2F{season}-{league_name_long}&div=div_stats_keeper'
            df_keepers = pd.read_html(url_keepers, header=1)[0]

            ##### Advanced Goalkeeper stats
            print(f'Scraping Advanced Goalkeeper stats...')
            url_keepers_adv = f'https://widgets.sports-reference.com/wg.fcgi?css=1&site=fb&url=%2Fen%2Fcomps%2F{league_name_short}%2F{season}%2Fkeepersadv%2Fplayers%2F{season}-{league_name_long}&div=div_stats_keeper_adv'
            df_keepers_adv = pd.read_html(url_keepers_adv, header=1)[0]

            ##### Playing Time stats
            print(f'Scraping Playing Time stats...')
            url_playing_time = f'https://widgets.sports-reference.com/wg.fcgi?css=1&site=fb&url=%2Fen%2Fcomps%2F{league_name_short}%2F{season}%2Fplayingtime%2Fplayers%2F{season}-{league_name_long}&div=div_stats_playing_time'
            df_playing_time = pd.read_html(url_playing_time, header=1)[0]

            ##### Miscellaneous stats
            print(f'Scraping Miscellaneous stats...')
            url_misc = f'https://widgets.sports-reference.com/wg.fcgi?css=1&site=fb&url=%2Fen%2Fcomps%2F{league_name_short}%2F{season}%2Fmisc%2Fplayers%2F{season}-{league_name_long}&div=div_stats_misc'
            df_misc = pd.read_html(url_misc, header=1)[0]

            ##### Concatenate defined individual DataFrames
                
            ####### Define DataFrames to be concatenated side-by-side (not all of them)
            lst_dfs = [df_keepers, df_keepers_adv]

            ###### Concatenate DataFrames side-by-side (indicated in list above)
            df_all = pd.concat(lst_dfs, axis=1)

            ###### Drop duplicate columns
            df_all = df_all.loc[:,~df_all.columns.duplicated()]

            ###### Drop duplicate rows
            df_all = df_all.drop_duplicates()
                
            ##### Left join defined individual DataFrames
                
            ####### Define join conditions
            conditions_join = ['Player', 'Nation', 'Pos', 'Squad', 'Comp']

            ###### Left join Standard Stats data
            df_all = pd.merge(df_all, df_std_stats, left_on=conditions_join, right_on=conditions_join, how='left')

            ###### Remove duplicate columns after join (contain '_y') and remove '_x' suffix from kept columns
            df_all = df_all[df_all.columns.drop(list(df_all.filter(regex='_y')))]
            df_all.columns = df_all.columns.str.replace('_x','')
                
            ###### Drop duplicate rows
            df_all = df_all.drop_duplicates()
                
            ###### Left join Playing Time data
            df_all = pd.merge(df_all, df_playing_time, left_on=conditions_join, right_on=conditions_join, how='left')

            ###### Remove duplicate columns after join (contain '_y') and remove '_x' suffix from kept columns
            df_all = df_all[df_all.columns.drop(list(df_all.filter(regex='_y')))]
            df_all.columns = df_all.columns.str.replace('_x','')
                
            ###### Drop duplicate rows
            df_all = df_all.drop_duplicates()

            ###### Left join Misc data
            df_all = pd.merge(df_all, df_misc, left_on=conditions_join, right_on=conditions_join, how='left')

            ###### Remove duplicate columns after join (contain '_y') and remove '_x' suffix from kept columns
            df_all = df_all[df_all.columns.drop(list(df_all.filter(regex='_y')))]
            df_all.columns = df_all.columns.str.replace('_x','')
                
            ###### Drop duplicate rows
            df_all = df_all.drop_duplicates()
                
            ##### FORMAT DataFrames
                
            ###### Take first two digits of age - fixes current season issue with extra values
            df_all['Age'] = df_all['Age'].astype(str).str[:2]
                
            ###### Create columns for league code and season
            df_all['League Name'] = league_name_long
            df_all['League ID'] = league_name_short
            df_all['Season'] = season              

             ###### Drop duplicates
            df_all = df_all.drop_duplicates()

    return df_all

# **5. Extracción de porteros**
Se hace una extracción por temporada para evitar el time out del servidor.

In [222]:
goalkeeperdata2022_2023 = get_fbref_goalkeeper_stats(lst_league_names,['2022-2023'])

Scraping started for the 2022-2023 season...
Scraping started for goalkeeper stats data for Big-5-European-Leagues league for the 2022-2023 season...
Scraping Standard stats...
Scraping Goalkeeper stats...
Scraping Advanced Goalkeeper stats...
Scraping Playing Time stats...
Scraping Miscellaneous stats...


In [223]:
playerdata2022_2023.drop(playerdata2022_2023.loc[playerdata2022_2023.Nation != 'es ESP', ].index, inplace = True)

In [224]:
goalkeeperdata2022_2023.drop(goalkeeperdata2022_2023.loc[goalkeeperdata2022_2023.Nation != 'es ESP', ].index, inplace = True)

In [225]:
'''Esta función sirve para decodificar los datos procedentes del html, suelen traer caracteres raros, hay que tener cuidado con ciertos caracteres ya que a pesar el encoding tienen tamaños en bytes raros
 y no se puede realizar una decodificación exacta, preguntar a Rubén como hacer esta limpieza de manera mas robusta'''

def decode_names(columnName, df):
  clean_names = []
  try:
    for elem in df[f'{columnName}']:
        elem = elem.encode('ISO-8859-1', 'ignore').decode("utf-8", 'ignore')
        clean_names.append(elem)
  except:
    print(f'esto no se puede decodificar: {elem}')

  return clean_names

In [226]:
playerdata2022_2023['Player'] = decode_names('Player', playerdata2022_2023)
playerdata2022_2023['Squad'] = decode_names('Squad', playerdata2022_2023)

In [227]:
goalkeeperdata2022_2023['Player'] = decode_names('Player', goalkeeperdata2022_2023)
goalkeeperdata2022_2023['Squad'] = decode_names('Squad', goalkeeperdata2022_2023)

In [228]:
playerdata2022_2023.isnull().sum().sum()
playerdata2022_2023 = playerdata2022_2023.fillna(0)

In [229]:
goalkeeperdata2022_2023.isnull().sum().sum()
goalkeeperdata2022_2023 = goalkeeperdata2022_2023.fillna(0)

In [230]:
playerdata2022_2023

,Rk,Player,Nation,Pos,Squad,Comp,Age,Born,MP,Starts,...,PKwon,PKcon,OG,Recov,Won,Lost,Won%,League Name,League ID,Season
24,25,Álvaro Aguado,es ESP,MF,Valladolid,es La Liga,26,1996,5,4,...,0,0,0,31,1,2,33.3,Big-5-European-Leagues,Big5,2022-2023
124,38,Sergio Akieme,es ESP,DF,Almería,es La Liga,24,1997,6,6,...,0,0,0,44,2,6,25.0,Big-5-European-Leagues,Big5,2022-2023
132,46,Jordi Alba,es ESP,DF,Barcelona,es La Liga,33,1989,3,1,...,0,0,0,18,3,1,75.0,Big-5-European-Leagues,Big5,2022-2023
133,47,Luis Alberto,es ESP,MF,Lazio,it Serie A,29,1992,7,2,...,0,0,0,23,3,2,60.0,Big-5-European-Leagues,Big5,2022-2023
134,48,Raúl Albiol,es ESP,DF,Villarreal,es La Liga,37,1985,6,6,...,0,0,0,47,19,7,73.1,Big-5-European-Leagues,Big5,2022-2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2224,2138,Gerard Yepes,es ESP,FW,Sampdoria,it Serie A,20,2002,1,0,...,0,0,0,1,0,0,0,Big-5-European-Leagues,Big5,2022-2023
2235,2149,Joseba Zaldúa,es ESP,DF,Cádiz,es La Liga,30,1992,6,5,...,0,0,0,39,11,9,55.0,Big-5-European-Leagues,Big5,2022-2023
2243,2157,Oier Zarraga,es ESP,MF,Athletic Club,es La Liga,23,1999,4,0,...,0,0,0,12,2,2,50.0,Big-5-European-Leagues,Big5,2022-2023
2259,2173,Igor Zubeldia,es ESP,DF,Real Sociedad,es La Liga,25,1997,6,5,...,0,0,0,48,8,11,42.1,Big-5-European-Leagues,Big5,2022-2023


In [231]:
playerdata2022_2023['Player'].duplicated().sum()

6

In [232]:
playerdata2022_2023.loc[playerdata2022_2023.Player == 'Carlos Soler', ]


,Rk,Player,Nation,Pos,Squad,Comp,Age,Born,MP,Starts,...,PKwon,PKcon,OG,Recov,Won,Lost,Won%,League Name,League ID,Season
1971,1885,Carlos Soler,es ESP,FW,Paris S-G,fr Ligue 1,25,1997,1,0,...,0,0,0,2,0,0,0,Big-5-European-Leagues,Big5,2022-2023
1972,1886,Carlos Soler,es ESP,MF,Valencia,es La Liga,25,1997,3,3,...,0,0,0,10,0,1,0.0,Big-5-European-Leagues,Big5,2022-2023


In [240]:
'''Me creo un Dataframe con lo mas basico sobre un jugador para mostrarlo en la api y probar que funciona, en un futuro se añadiran las estadisticas que sea necesarias.'''
basicPlayer = pd.DataFrame()
basicPlayer['Player'] = playerdata2022_2023['Player'] 
basicPlayer['Nation'] = playerdata2022_2023['Nation'] 
basicPlayer['Pos'] = playerdata2022_2023['Pos'] 
basicPlayer['Squad'] = playerdata2022_2023['Squad'] 
basicPlayer['Comp'] = playerdata2022_2023['Comp'] 
basicPlayer['Age'] = playerdata2022_2023['Age'] 
basicPlayer.reset_index(drop=True, inplace=True)

In [242]:
'''Me creo un Dataframe con lo mas basico sobre un jugador para mostrarlo en la api y probar que funciona, en un futuro se añadiran las estadisticas que sea necesarias.'''
basicGoalKeeper = pd.DataFrame()
basicGoalKeeper['Player'] = goalkeeperdata2022_2023['Player'] 
basicGoalKeeper['Nation'] = goalkeeperdata2022_2023['Nation'] 
basicGoalKeeper['Pos'] = goalkeeperdata2022_2023['Pos'] 
basicGoalKeeper['Squad'] = goalkeeperdata2022_2023['Squad'] 
basicGoalKeeper['Comp'] = goalkeeperdata2022_2023['Comp'] 
basicGoalKeeper['Age'] = goalkeeperdata2022_2023['Age'] 
basicGoalKeeper.reset_index(drop=True, inplace=True)

In [244]:
basicPlayer.to_json('basicPlayer.json',orient='table')

In [245]:
basicGoalKeeper.to_json('basicGK.json',orient='table')

Cosas pendientes:
* Añadir mas ligas




* Añadir mas filtros
* Crear dfs segun posicion
* Duplicados: puede ser que algun jugador empiece la temporada en un equipo y en algun punto de la misma se cambie a otro equipo, sumar las columnas que se puedan sumar y quedarme con el ultimo equipo con los datos sumados o alguna solucion similar.
* Nation dejar solo un valor o es o ESP. Separar los valores de Comp en Comp y en Nation Comp, separar la liga del pais en el que se juega esa liga
* Añadir graficos, recomendador de jugadore similares, etc
* Paralelizar extaccion de datos
* Modularizar el codigo para la extraccion de porteros y jugadores.

